In [1]:
from joblib import load
import pandas as pd
import serial
import time
from twilio.rest import Client
import twilio_numbers

client = Client(twilio_numbers.account_sid,twilio_numbers.auth_token)


# Loading the model from disk
filename = 'simmulation_model.joblib'
loaded_model = load(filename)
kmeans = load('kmeans.joblib')

SerialObj = serial.Serial('COM3')
#SerialObj.open()


print('\n current settings')
print('Port =',SerialObj.port)
print ('Baudrate = ',SerialObj.baudrate)
print ('Bytesize = ',SerialObj.bytesize)
print ('Parity = ',SerialObj.parity)
print ('stopBits = ',SerialObj.stopbits)

SerialObj.timeout = 3
time.sleep(3)

received_data = []

while True:
    if SerialObj.in_waiting:
        RecievedString = SerialObj.readline()
        received_data.append(RecievedString.decode('utf'))
        print(received_data[-1])  # print the last received data
        if 'end' in received_data[-1]:  # stop the loop once the newline character is received
            break

SerialObj.close()


# Assume these are your new inputs for which you want to predict severity
new_inputs = {
    'vibration_value': received_data[4],
    'temperature': received_data[0],
    'burst_in_x': received_data[1],
    'burst_in_y': received_data[2],
    'burst_in_z': received_data[3],
    'roll':57,  # this value is hard coded since adding more pins on a simulation is a bit complex
    'pitch':22, # this value is hard coded since adding more pins on a simulation is a bit complex
    
    
}

# Convert the inputs to a DataFrame
new_data = pd.DataFrame([new_inputs])

# Scale the new data
#new_data_scaled = scaler.transform(new_data)

# Apply KMeans clustering to the new data
new_data['cluster'] = kmeans.predict(new_data)

# Use the trained model to predict severity
new_severity = loaded_model.predict(new_data)

# data from the GPS module
latitude = -29.45132
longitude = 27.72361

google_maps_link = f"https://www.google.com/maps/search/?api=1&query={latitude},{longitude}"

alert_message = f"""
EMERGENCY ALERT: A potential vehicle collision has been detected. Details are as follows:
- Location:{google_maps_link}
- Severity: {new_severity[0]}
- Time of incident: {received_data[6]}
- The number of passengers: {received_data[5]}
Immediate assistance is required. Please dispatch emergency services to the location immediately.
"""




message = client.messages.create(
body = alert_message,
    from_ = twilio_numbers.twilio_number,
    to = twilio_numbers.target_number
)


print("*****************CADNS ALERT****************************************")

print('Temperature: '+ received_data[0])
print('Burst in x_axis:' + received_data[1])
print('Burst in y_axis:' + received_data[2])
print('Burst in z_axis:' + received_data[3])
print('Vibration Value:' + received_data[4])
print('The number of passengers is:' + received_data[5])
print('Time of accident is: '+  received_data[3])


print(f'Predicted Severity: {new_severity[0]}')
print("**********************************************************************")
print(alert_message)



 current settings
Port = COM3
Baudrate =  9600
Bytesize =  8
Parity =  N
stopBits =  1



KeyboardInterrupt

